In [59]:
import geopandas as gpd
import numpy as np
import pandas as pd
import aseg_gdf2


In [60]:
infile = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\DR\2017_DalyRiver_SkyTEM\inversion_ready\DR_inversion_ready"

dat = aseg_gdf2.read(infile)

In [62]:
df_AEM = dat.df()

# Convert to geopandas data frame
gdf_AEM = gpd.GeoDataFrame(df_AEM, 
                           geometry=gpd.points_from_xy(df_AEM['easting '],
                                                       df_AEM['northing']))

In [65]:
# Bring in the polygon data for subseting

infile = r"C:\Users\PCUser\Desktop\NSC_data\data\vector\DalyRiver\Jinduckin_extent.shp"

gdf = gpd.read_file(infile)

# Convert to easting northing
#gdf = gdf.to_crs({'init': 'epsg:28352'})

poly = gdf['geometry'].iloc[-1]

In [69]:
gdf_AEM['inside_poly'] = gdf_AEM.within(poly.buffer(1000))

In [70]:
# Now we get the indices of points that are inside the poly

gdf_subset = gdf_AEM[gdf_AEM['inside_poly'] == True]

In [79]:
gdf_subset["line "]

40841     102701
40842     102701
40843     102701
40844     102701
40845     102701
           ...  
236956    109204
236957    109204
236958    109204
236959    109204
236960    109204
Name: line , Length: 44488, dtype: int64

In [78]:
# Lets just keep certain lines

lines = [106301, 109201, 109202, 109203, 109204, 103501, 105601, 105602, 102701, 109701, 110301, 110102, 110203]

gdf_subset = gdf_AEM[gdf_AEM['line '].isin(lines)]

In [81]:
# Now we write out our data into a text file using the indices of the dataframe
infile =  r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\DR\2017_DalyRiver_SkyTEM\inversion_ready\DR_inversion_ready.dat"
outfile =  r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\DR\2017_DalyRiver_SkyTEM\inversion_ready\DR_inversion_ready_NS_subset.dat"

inf = open(infile)
lines = inf.readlines()

with open(outfile, 'w') as outf:
    for item in gdf_subset.index.tolist():
        outf.write(lines[item])
        